## Modelling

In [3]:
#Importing Required Libraries
import numpy as np
import pandas as pd
import dask.dataframe as dk
import calendar
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler,Normalizer
from scipy.sparse import csr_matrix,hstack
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import pickle
import tensorflow as tf

2025-05-03 04:16:12.889723: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-03 04:16:13.121136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746245773.141345   13514 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746245773.147471   13514 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746245773.162778   13514 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
dtype={'id'       :     'object', 
    'item_id'     :  'int64', 
    'dept_id'     :  'int8', 
    'cat_id'      :  'int8', 
    'store_id'    :  'int8', 
    'state_id'    :  'int8', 
    'd'           :  'object', 
    'sales'       :  'int16',  
    'date'        : 'object', 
   'wday'        :  'int8',  
   'month'       :  'int8',  
   'year'        :  'int16',  
   'event_name_1' : 'int8', 
   'event_type_1' : 'int8', 
   'event_name_2' : 'int8', 
   'event_type_2' : 'int8', 
    'snap':'int8',
  'sell_price'   : 'float16',
       'price_change':'float16',
   'week_number'  : 'int8',  
   'season'       : 'object', 
   'quater_start' : 'int8',  
   'quater_end'   : 'int8',  
   'month_start'  : 'int8',  
   'month_end'    : 'int8',  
   'year_start'   : 'int8',  
   'year_end'     : 'int8',  
   'group'        : 'int8',  
   'no_events'    : 'object', 
   'holiday'      : 'object',
    'week_number':'int8',
       'season':'int8',
       'quater_start':'int8',
       'quater_end':'int8',
       'month_start':'int8',
       'month_end':'int8',
       'year_start':'int8',
       'year_end':'int8',
       'roll_7_shift_28_mean':'float16',
       'roll_14_shift_28_mean':'float16',
       'roll_30_shift_28_mean':'float16',
       'roll_60_shift_28_mean':'float16',
       'roll_360_shift_28_mean':'float16',
       'roll_7_shift_28_std':'float16',
       'roll_14_shift_28_std':'float16',
       'roll_30_shift_28_std':'float16',
       'roll_60_shift_28_std':'float16',
       'roll_360_shift_28_std':'float16',
       'direct_ewm':'float16',
       'direct_lag_28':'int16',
       'direct_lag_35':'int16',
       'direct_lag_42':'int16',
       'direct_lag_49':'int16',
       'direct_lag_56':'int16',
       'direct_lag_63':'int16',
       'direct_lag_70':'int16',
       'direct_lag_77':'int16',
       'direct_lag_84':'int16',
       'direct_lag_91':'int16',
       'direct_lag_98':'int16',
       'min_price':'float16',
       'max_price':'float16',
       'mean_price':'float16',
       'std_price':'float16',
       'price_norm_1':'float16',
       'price_norm_2':'float16',
       'price_norm_3':'float16',
      }

In [5]:
train=pd.read_feather('Intermediate Data/train1.feather')
cv=pd.read_feather('Intermediate Data/cv1.feather')
test=pd.read_feather('Intermediate Data/test1.feather')
final_test=pd.read_feather('Intermediate Data/final_test1.feather')

In [6]:
for col, typ in dtype.items():
    if col in train.columns:
        train[col] = train[col].astype(typ)
        cv[col] = cv[col].astype(typ)
        test[col] = test[col].astype(typ)
        final_test[col] = final_test[col].astype(typ)

In [7]:
# WRMSSE Calculation
def caluclate_WRMSSE(actual,predicted,train,weights,h,n):
    '''This function is used to calculate RMSSE'''
    num=((actual-predicted)**2).sum(axis=1)/h
    denom=(train[:,1:]-train[:,:-1])**2
    denom=denom.sum(axis=1)/(n-1)
    return (num/denom)**0.5

In [20]:
def get_model_performances(model,train,cv,test,X_cv,X_test):
    '''This Function is used to get WRMSSE that is used in this Case Study as a Metric For CV and Test Data'''
    #For CV Data
    cv['prices']=cv['sales']*cv['sell_price']
    total_sales=cv.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i]=total_sales.loc[i]['sales']/total
    train1=pd.concat([train,cv])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    cv['pred_sales']=model.predict(X_cv)
    df1=cv.pivot_table(index=['id'],columns='d',values='pred_sales')
    dic={}
    for j,i in enumerate(range(1886,1914)):
        dic['d_'+str(i)]='F'+str(j+1)
    df1=df1.rename(columns=dic) 
    df1.reset_index(level=[0],inplace=True)
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1914)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891', 'd_1892',
       'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898', 'd_1899',
       'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1886)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1886)
    cv_wrsmme=np.sum(rmsse*weights)
    cv.drop(['pred_sales'],axis=1,inplace=True)
    del actual,predicted,training,weights,agg
    
    #For Test data
    test['prices']=test['sales']*test['sell_price']
    total_sales=test.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i.replace('evaluation','validation')]=total_sales.loc[i]['sales']/total
    test['id']=test['id'].apply(lambda x:x.replace('evaluation','validation'))
    train1=pd.concat([train,cv,test])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    test['pred_sales']=model.predict(X_test)
    df1=test.pivot_table(index=['id'],columns='d',values='pred_sales')
    dic={}
    for j,i in enumerate(range(1914,1942)):
        dic['d_'+str(i)]='F'+str(j+1)
    df1=df1.rename(columns=dic) 
    df1.reset_index(level=[0],inplace=True)
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1942)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919',
       'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925',
       'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931',
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937',
       'd_1938', 'd_1939', 'd_1940', 'd_1941']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1914)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1914)
    test_wrsmme=np.sum(rmsse*weights)

    print("CV WRMSSE=",cv_wrsmme)
    print("Test WRMSSE=",test_wrsmme)

In [9]:
def get_model_performances_for_store_wise_trained_model(train,cv,test,cv_pred,test_pred):
    '''This Function is used to get WRMSSE that is used in this Case Study as a Metric For CV and Test Data where model is trained according to store id(Mainly used for Catabosst)'''
    #For CV Data
    cv['prices']=cv['sales']*cv['sell_price']
    total_sales=cv.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i]=total_sales.loc[i]['sales']/total
    train1=pd.concat([train,cv])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    
    df1=cv_pred
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1914)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1886', 'd_1887', 'd_1888', 'd_1889', 'd_1890', 'd_1891', 'd_1892',
       'd_1893', 'd_1894', 'd_1895', 'd_1896', 'd_1897', 'd_1898', 'd_1899',
       'd_1900', 'd_1901', 'd_1902', 'd_1903', 'd_1904', 'd_1905', 'd_1906',
       'd_1907', 'd_1908', 'd_1909', 'd_1910', 'd_1911', 'd_1912', 'd_1913']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1886)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1886)
    cv_wrsmme=np.sum(rmsse*weights)
    del actual,predicted,training,weights,agg
    
    #For Test data
    test['prices']=test['sales']*test['sell_price']
    total_sales=test.groupby('id').sum()
    total=sum(total_sales['sales'])
    weight={}
    for i in total_sales.index:
        weight[i.replace('evaluation','validation')]=total_sales.loc[i]['sales']/total
    test['id']=test['id'].apply(lambda x:x.replace('evaluation','validation'))
    train1=pd.concat([train,cv,test])
    train1.sort_values(['id','date'],inplace=True)
    train1.fillna(0,inplace=True)
    df=train1.pivot_table(index=['id','state_id','store_id','cat_id','dept_id','item_id'],columns='d',values='sales')
    df.reset_index(level=[0,1,2,3,4,5],inplace=True)
    df.fillna(0,inplace=True)
    del train1
    import gc
    df1=test_pred
    df1['id']=df1['id'].apply(lambda x:x.replace('evaluation','validation'))
    dd=df.merge(df1,on='id')
    dd['weight']=dd['id'].apply(lambda x:weight[x])
    l=['d_'+str(i) for i in range(1,1942)]
    l1=['F'+str(i) for i in range(1,29)]
    agg_level={2:['state_id'],3:['store_id'],4:['cat_id'],5:['dept_id'],6:['state_id','cat_id'],\
           7:['state_id','dept_id'],8:['store_id','cat_id'],9:['store_id','dept_id'],10:['item_id'],11:['item_id','state_id']}
    agg=pd.DataFrame(dd[l+l1].sum()).transpose()
    agg['weight']=1/12
    agg['level']=1
    col=agg.columns
    for level in agg_level:
        temp_df=dd.groupby(by=agg_level[level]).sum().reset_index(drop=True)
        temp_df['weight']/=12
        temp_df['level']=level
        agg = pd.concat([agg, temp_df[col]], ignore_index=True)
    dd['weight']/=12
    dd['level']=12    
    agg = pd.concat([agg, dd[col]], ignore_index=True)
    actual=agg[['d_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919',
       'd_1920', 'd_1921', 'd_1922', 'd_1923', 'd_1924', 'd_1925',
       'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931',
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937',
       'd_1938', 'd_1939', 'd_1940', 'd_1941']].values
    predicted=agg[['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']].values
    training=agg[['d_'+str(i) for i in range(1,1914)]].values
    weights=agg['weight'].values
    rmsse=caluclate_WRMSSE(actual,predicted,training,weights,28,1914)
    test_wrsmme=np.sum(rmsse*weights)

    print("CV WRMSSE=",cv_wrsmme)
    print("Test WRMSSE=",test_wrsmme)

### LSTM Model

In [11]:
tf.keras.backend.clear_session()
input1 = tf.keras.layers.Input(shape=(1,), name='Item_Id')
input2 = tf.keras.layers.Input(shape=(1,), name='Dept_Id')
input3 = tf.keras.layers.Input(shape=(1,), name='Cat_Id')
input4 = tf.keras.layers.Input(shape=(1,), name='Store_Id')
input5 = tf.keras.layers.Input(shape=(1,), name='State_Id')
input6 = tf.keras.layers.Input(shape=(1,), name='year')
input7 = tf.keras.layers.Input(shape=(1,), name='event_name_1')
input8 = tf.keras.layers.Input(shape=(1,), name='event_name_2')
input9 = tf.keras.layers.Input(shape=(1,), name='season')
input10 = tf.keras.layers.Input(shape=(1, 23), name='Numerical_features')

emb1=tf.keras.layers.Embedding(3050,output_dim=150)(input1)
emb2=tf.keras.layers.Embedding(8,output_dim=10)(input2)
emb3=tf.keras.layers.Embedding(4,output_dim=10)(input3)
emb4=tf.keras.layers.Embedding(11,output_dim=10)(input4)
emb5=tf.keras.layers.Embedding(4,output_dim=10)(input5)
emb6=tf.keras.layers.Embedding(2017,output_dim=10)(input6)
emb7=tf.keras.layers.Embedding(32,output_dim=10)(input7)
emb8=tf.keras.layers.Embedding(6,output_dim=10)(input8)
emb9=tf.keras.layers.Embedding(5,output_dim=10)(input9)

lstm1=tf.keras.layers.LSTM(50)(emb1)
lstm2=tf.keras.layers.LSTM(10)(emb2)
lstm3=tf.keras.layers.LSTM(10)(emb3)
lstm4=tf.keras.layers.LSTM(10)(emb4)
lstm5=tf.keras.layers.LSTM(10)(emb5)
lstm6=tf.keras.layers.LSTM(10)(emb6)
lstm7=tf.keras.layers.LSTM(10)(emb7)
lstm8=tf.keras.layers.LSTM(10)(emb8)
lstm9=tf.keras.layers.LSTM(10)(emb9)
lstm10=tf.keras.layers.LSTM(10)(input10)

x1=tf.keras.layers.Flatten()(lstm1)
x2=tf.keras.layers.Flatten()(lstm2)
x3=tf.keras.layers.Flatten()(lstm3)
x4=tf.keras.layers.Flatten()(lstm4)
x5=tf.keras.layers.Flatten()(lstm5)
x6=tf.keras.layers.Flatten()(lstm6)
x7=tf.keras.layers.Flatten()(lstm7)
x8=tf.keras.layers.Flatten()(lstm8)
x9=tf.keras.layers.Flatten()(lstm9)
x10=tf.keras.layers.Flatten()(input10)

x=tf.keras.layers.Concatenate()([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10])
x=tf.keras.layers.BatchNormalization()(x)
x=tf.keras.layers.Dense(256,activation='sigmoid')(x)
x=tf.keras.layers.Dense(128,activation='tanh')(x)
x=tf.keras.layers.Dense(64,activation='relu')(x)
x=tf.keras.layers.Dense(1)(x)
model=tf.keras.Model([input1,input2,input3,input4,input5,input6,input7,input8,input9,input10],x)

W0000 00:00:1746245968.666501   13514 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.0009),loss='mse')

In [15]:
model.fit(
    x=[
        train['item_id'].values.reshape(-1,1),
        train['dept_id'].values.reshape(-1,1),
        train['cat_id'].values.reshape(-1,1),
        train['store_id'].values.reshape(-1,1),
        train['state_id'].values.reshape(-1,1),
        train['year'].values.reshape(-1,1),
        train['event_name_1'].values.reshape(-1,1),
        train['event_name_2'].values.reshape(-1,1),
        train['season'].values.reshape(-1,1),
        train[
            [
                'roll_7_shift_28_mean', 'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
                'roll_60_shift_28_mean', 'roll_360_shift_28_mean', 'roll_7_shift_28_std',
                'roll_14_shift_28_std', 'roll_30_shift_28_std', 'roll_60_shift_28_std',
                'roll_360_shift_28_std', 'sell_price', 'direct_lag_28', 'direct_lag_35',
                'direct_lag_42', 'direct_lag_49', 'direct_lag_56', 'direct_lag_63',
                'direct_lag_70', 'direct_lag_77', 'direct_lag_84', 'direct_lag_91',
                'direct_lag_98', 'direct_ewm'
            ]
        ].values.reshape(-1,1,23)
    ],
    y=train['sales'],
    validation_data=(
        [
            cv['item_id'].values.reshape(-1,1),
            cv['dept_id'].values.reshape(-1,1),
            cv['cat_id'].values.reshape(-1,1),
            cv['store_id'].values.reshape(-1,1),
            cv['state_id'].values.reshape(-1,1),
            cv['year'].values.reshape(-1,1),
            cv['event_name_1'].values.reshape(-1,1),
            cv['event_name_2'].values.reshape(-1,1),
            cv['season'].values.reshape(-1,1),
            cv[
                [
                    'roll_7_shift_28_mean', 'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
                    'roll_60_shift_28_mean', 'roll_360_shift_28_mean', 'roll_7_shift_28_std',
                    'roll_14_shift_28_std', 'roll_30_shift_28_std', 'roll_60_shift_28_std',
                    'roll_360_shift_28_std', 'sell_price', 'direct_lag_28', 'direct_lag_35',
                    'direct_lag_42', 'direct_lag_49', 'direct_lag_56', 'direct_lag_63',
                    'direct_lag_70', 'direct_lag_77', 'direct_lag_84', 'direct_lag_91',
                    'direct_lag_98', 'direct_ewm'
                ]
            ].values.reshape(-1,1,23)
        ],
        cv['sales']
    ),
    batch_size=1000000,
    epochs=15,
    verbose=1
)

Epoch 1/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - loss: 6.8024 - val_loss: 4.8700
Epoch 2/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - loss: 6.7789 - val_loss: 4.9005
Epoch 3/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - loss: 6.7771 - val_loss: 4.9070
Epoch 4/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - loss: 6.7270 - val_loss: 4.9074
Epoch 5/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - loss: 6.7206 - val_loss: 4.8924
Epoch 6/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 213s 4s/step - loss: 6.6850 - val_loss: 4.9062
Epoch 7/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 228s 4s/step - loss: 6.7096 - val_loss: 4.8993
Epoch 8/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - loss: 6.6655 - val_loss: 4.9286
Epoch 9/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - loss: 6.7662 - val_loss: 4.8954
Epoch 10/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 196s 4s/step - loss: 6.6566 - val_loss: 4.9137
Epoch 11/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - loss: 6.6790 - val_loss: 4.9258
Epoch 12/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 193s 4s/step - loss: 6.5

In [21]:
get_model_performances(model,train,cv,test,[cv['item_id'].values.reshape(-1,1),cv['dept_id'].values,cv['cat_id'].values.reshape(-1,1),cv['store_id'].values.reshape(-1,1),\
             cv['state_id'].values.reshape(-1,1),cv['year'].values.reshape(-1,1),cv['event_name_1'].values.reshape(-1,1),cv['event_name_2'].values.reshape(-1,1),\
             cv['season'].values.reshape(-1,1),cv[['roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','direct_ewm']].values.reshape(-1,1,23)],\
                       [test['item_id'].values.reshape(-1,1),test['dept_id'].values,test['cat_id'].values.reshape(-1,1),test['store_id'].values.reshape(-1,1),\
             test['state_id'].values.reshape(-1,1),test['year'].values.reshape(-1,1),test['event_name_1'].values.reshape(-1,1),test['event_name_2'].values.reshape(-1,1),\
             test['season'].values.reshape(-1,1),test[['roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','direct_ewm']].values.reshape(-1,1,23)])

26679/26679 ━━━━━━━━━━━━━━━━━━━━ 58s 2ms/step
26679/26679 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step
CV WRMSSE= 0.9158538873602474
Test WRMSSE= 0.8622420347782332


In [19]:
model.save('lstm_model.keras') 

### Submission

In [22]:
test['pred_sales']=model.predict([test['item_id'].values.reshape(-1,1),test['dept_id'].values,test['cat_id'].values.reshape(-1,1),test['store_id'].values.reshape(-1,1),\
             test['state_id'].values.reshape(-1,1),test['year'].values.reshape(-1,1),test['event_name_1'].values.reshape(-1,1),test['event_name_2'].values.reshape(-1,1),\
             test['season'].values.reshape(-1,1),test[['roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','direct_ewm']].values.reshape(-1,1,23)])

26679/26679 ━━━━━━━━━━━━━━━━━━━━ 56s 2ms/step


In [23]:
final_test['pred_sales']=model.predict([final_test['item_id'].values.reshape(-1,1),test['dept_id'].values,final_test['cat_id'].values.reshape(-1,1),final_test['store_id'].values.reshape(-1,1),\
             final_test['state_id'].values.reshape(-1,1),final_test['year'].values.reshape(-1,1),final_test['event_name_1'].values.reshape(-1,1),final_test['event_name_2'].values.reshape(-1,1),\
             final_test['season'].values.reshape(-1,1),final_test[['roll_7_shift_28_mean',
       'roll_14_shift_28_mean', 'roll_30_shift_28_mean',
       'roll_60_shift_28_mean', 'roll_360_shift_28_mean',
       'roll_7_shift_28_std', 'roll_14_shift_28_std', 'roll_30_shift_28_std',
       'roll_60_shift_28_std', 'roll_360_shift_28_std','sell_price','direct_lag_28', 'direct_lag_35', 'direct_lag_42', 'direct_lag_49',
       'direct_lag_56', 'direct_lag_63', 'direct_lag_70', 'direct_lag_77',
       'direct_lag_84', 'direct_lag_91', 'direct_lag_98','direct_ewm']].values.reshape(-1,1,23)])

26679/26679 ━━━━━━━━━━━━━━━━━━━━ 60s 2ms/step


In [24]:
df=test.pivot_table(index='id',columns='date',values='pred_sales')
df.reset_index(level=0,inplace=True)
df['id']=df['id'].apply(lambda x:x.replace('evaluation','validation'))
dic={}
l=test['date'].unique()
for i,day in enumerate(l):
    dic[day]='F'+str(i+1)
df.rename(columns=dic,inplace=True) 

In [25]:
df1=final_test.pivot_table(index='id',columns='d',values='pred_sales')
df1.reset_index(level=0,inplace=True)
dic={}
for i,day in enumerate(range(1942,1970)):
    dic['d_'+str(day)]='F'+str(i+1)
df1.rename(columns=dic,inplace=True)

In [26]:
df2=pd.concat([df1,df])

In [27]:
df2.to_csv('lstm_submission_1.csv',index=False)